In [15]:
import numpy as np
import pandas as pd

In [16]:
# import options data
data = pd.read_csv("NDXP.csv").sort_values(by=['quote_date', 'strike'])
data

,quote_date,expiration,strike,option_type,bid_1545,ask_1545
10542,2018-01-04,2018-02-02,6200.0,C,401.9,423.0
10543,2018-01-04,2018-02-02,6200.0,P,11.8,16.0
10580,2018-01-04,2018-02-09,6200.0,C,406.2,427.0
10581,2018-01-04,2018-02-09,6200.0,P,18.7,23.0
10618,2018-01-04,2018-02-23,6200.0,C,417.5,439.0
...,...,...,...,...,...,...
3261,2022-12-30,2023-01-06,16000.0,P,5106.0,5122.1
5600,2022-12-30,2023-01-13,16000.0,C,0.0,0.9
5601,2022-12-30,2023-01-13,16000.0,P,5089.8,5106.2
7656,2022-12-30,2023-01-27,16000.0,C,0.0,1.2


In [17]:
# import interest rate data
rates = pd.read_csv("DGS1MO.csv").ffill()
rates.replace('.', pd.NA, inplace=True)
rates.columns = ['date','rate']
rates['date'] = pd.to_datetime(rates['date'])
rates['rate'] = (pd.to_numeric(rates['rate'])/100).fillna(method='ffill')
rates

,date,rate
0,2018-01-02,0.0129
1,2018-01-03,0.0129
2,2018-01-04,0.0128
3,2018-01-05,0.0127
4,2018-01-08,0.0130
...,...,...
1299,2022-12-26,0.0380
1300,2022-12-27,0.0387
1301,2022-12-28,0.0386
1302,2022-12-29,0.0404


In [18]:
# second fridays between 2018 and 2022

all_fridays = pd.date_range(start="2018-01-01", end="2022-12-31", freq='W-FRI') ## all fridays in the period

second_fridays = [] # initialize second fridays list

for friday in all_fridays: # loop through every friday and determine if it is second friday
    month_start = pd.offsets.MonthBegin(1) # create a date offset to move friday to start of the month
    if len(pd.date_range(start = friday - month_start, end = friday, freq='W-FRI')) == 2: # if number of fridays since start of month is 2:
        second_fridays.append(friday)

second_fridays = pd.Series(second_fridays)
second_fridays

0    2018-01-12
1    2018-02-09
2    2018-03-09
3    2018-04-13
4    2018-05-11
5    2018-06-08
6    2018-07-13
7    2018-08-10
8    2018-09-14
9    2018-10-12
10   2018-11-09
11   2018-12-14
12   2019-01-11
13   2019-02-08
14   2019-03-08
15   2019-04-12
16   2019-05-10
17   2019-06-14
18   2019-07-12
19   2019-08-09
20   2019-09-13
21   2019-10-11
22   2019-11-08
23   2019-12-13
24   2020-01-10
25   2020-02-14
26   2020-03-13
27   2020-04-10
28   2020-05-08
29   2020-06-12
30   2020-07-10
31   2020-08-14
32   2020-09-11
33   2020-10-09
34   2020-11-13
35   2020-12-11
36   2021-01-08
37   2021-02-12
38   2021-03-12
39   2021-04-09
40   2021-05-14
41   2021-06-11
42   2021-07-09
43   2021-08-13
44   2021-09-10
45   2021-10-08
46   2021-11-12
47   2021-12-10
48   2022-01-14
49   2022-02-11
50   2022-03-11
51   2022-04-08
52   2022-05-13
53   2022-06-10
54   2022-07-08
55   2022-08-12
56   2022-09-09
57   2022-10-14
58   2022-11-11
59   2022-12-09
dtype: datetime64[ns]

In [19]:
# make sure dates are datetime format
data['expiration'] = pd.to_datetime(data['expiration'])
data['quote_date'] = pd.to_datetime(data['quote_date'])

# perform a left join to get second-friday expirations
second_fridays_series = pd.Series(second_fridays)
second_friday_data = data[data['expiration'].isin(second_fridays_series)]

# only get quotes that are one month before every expiration
second_friday_data = second_friday_data[(second_friday_data['expiration'] - second_friday_data['quote_date']).dt.days == 28]
second_friday_data

,quote_date,expiration,strike,option_type,bid_1545,ask_1545
12652,2018-01-12,2018-02-09,5400.0,C,1347.00,1368.30
12653,2018-01-12,2018-02-09,5400.0,P,0.05,3.10
12654,2018-01-12,2018-02-09,5500.0,C,1248.00,1269.00
12655,2018-01-12,2018-02-09,5500.0,P,0.25,3.70
12656,2018-01-12,2018-02-09,5600.0,C,1151.60,1165.90
...,...,...,...,...,...,...
6823188,2022-11-11,2022-12-09,14800.0,P,2939.00,2963.80
6823189,2022-11-11,2022-12-09,14900.0,C,0.00,2.10
6823190,2022-11-11,2022-12-09,14900.0,P,3038.60,3063.30
6823191,2022-11-11,2022-12-09,15000.0,C,0.00,1.95


In [20]:
# Make sure bid < ask ( buy < sell)
second_friday_data = second_friday_data[second_friday_data['bid_1545'] < second_friday_data['ask_1545']]
second_friday_data

,quote_date,expiration,strike,option_type,bid_1545,ask_1545
12652,2018-01-12,2018-02-09,5400.0,C,1347.00,1368.30
12653,2018-01-12,2018-02-09,5400.0,P,0.05,3.10
12654,2018-01-12,2018-02-09,5500.0,C,1248.00,1269.00
12655,2018-01-12,2018-02-09,5500.0,P,0.25,3.70
12656,2018-01-12,2018-02-09,5600.0,C,1151.60,1165.90
...,...,...,...,...,...,...
6823188,2022-11-11,2022-12-09,14800.0,P,2939.00,2963.80
6823189,2022-11-11,2022-12-09,14900.0,C,0.00,2.10
6823190,2022-11-11,2022-12-09,14900.0,P,3038.60,3063.30
6823191,2022-11-11,2022-12-09,15000.0,C,0.00,1.95


In [21]:
# organise puts and calls for each strike and date
calls = second_friday_data[second_friday_data['option_type'] == "C"]
puts = second_friday_data[second_friday_data['option_type'] == "P"]
merged_data = pd.merge(calls, puts, on=['quote_date', 'strike'], suffixes=('_C','_P'), how='inner')

# get 1 month rates on each date
merged_data = pd.merge(merged_data, rates, left_on='quote_date', right_on='date', how='left')

# use put call parity to get underlying price
merged_data['implied_S'] = (merged_data['bid_1545_C'] + merged_data['ask_1545_C'])/2 + merged_data['strike']/((1+merged_data['rate'])**(1/12)) - (merged_data['bid_1545_P'] + merged_data['ask_1545_P'])/2
merged_data['moneyness'] = merged_data['implied_S'] - merged_data['strike']
merged_data

,quote_date,expiration_C,strike,option_type_C,bid_1545_C,ask_1545_C,expiration_P,option_type_P,bid_1545_P,ask_1545_P,date,rate,implied_S,moneyness
0,2018-01-12,2018-02-09,5400.0,C,1347.00,1368.30,2018-02-09,P,0.05,3.1,2018-01-12,0.0131,6750.221453,1350.221453
1,2018-01-12,2018-02-09,5500.0,C,1248.00,1269.00,2018-02-09,P,0.25,3.7,2018-01-12,0.0131,6750.563054,1250.563054
2,2018-01-12,2018-02-09,5600.0,C,1151.60,1165.90,2018-02-09,P,0.65,4.2,2018-01-12,0.0131,6750.254655,1150.254655
3,2018-01-12,2018-02-09,5700.0,C,1052.40,1066.70,2018-02-09,P,1.70,4.3,2018-01-12,0.0131,6750.371256,1050.371256
4,2018-01-12,2018-02-09,5800.0,C,953.10,967.40,2018-02-09,P,2.40,5.1,2018-01-12,0.0131,6750.212857,950.212857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10893,2022-11-11,2022-12-09,14600.0,C,0.20,2.60,2022-12-09,P,2740.00,2764.7,2022-11-11,0.0371,11804.796038,-2795.203962
10894,2022-11-11,2022-12-09,14700.0,C,0.10,2.40,2022-12-09,P,2839.40,2864.2,2022-11-11,0.0371,11804.892928,-2895.107072
10895,2022-11-11,2022-12-09,14800.0,C,0.05,2.25,2022-12-09,P,2939.00,2963.8,2022-11-11,0.0371,11804.889819,-2995.110181
10896,2022-11-11,2022-12-09,14900.0,C,0.00,2.10,2022-12-09,P,3038.60,3063.3,2022-11-11,0.0371,11804.936710,-3095.063290


In [22]:
# might need to consider picking a single implied S (maybe one closest to at the money?)
def get_at_the_money(group): 
    return group.iloc[(group['moneyness'].abs()).argmin()] # for each date, get most at the money strike
at_the_money_df = merged_data.groupby('quote_date').apply(get_at_the_money)[['quote_date', 'implied_S']] # pull out the calculated S for most at the money strike
at_the_money_df.reset_index(drop=True, inplace=True)
merged_data = pd.merge(merged_data, at_the_money_df, left_on='quote_date', right_on='quote_date', how='left')
merged_data

,quote_date,expiration_C,strike,option_type_C,bid_1545_C,ask_1545_C,expiration_P,option_type_P,bid_1545_P,ask_1545_P,date,rate,implied_S_x,moneyness,implied_S_y
0,2018-01-12,2018-02-09,5400.0,C,1347.00,1368.30,2018-02-09,P,0.05,3.1,2018-01-12,0.0131,6750.221453,1350.221453,6751.233067
1,2018-01-12,2018-02-09,5500.0,C,1248.00,1269.00,2018-02-09,P,0.25,3.7,2018-01-12,0.0131,6750.563054,1250.563054,6751.233067
2,2018-01-12,2018-02-09,5600.0,C,1151.60,1165.90,2018-02-09,P,0.65,4.2,2018-01-12,0.0131,6750.254655,1150.254655,6751.233067
3,2018-01-12,2018-02-09,5700.0,C,1052.40,1066.70,2018-02-09,P,1.70,4.3,2018-01-12,0.0131,6750.371256,1050.371256,6751.233067
4,2018-01-12,2018-02-09,5800.0,C,953.10,967.40,2018-02-09,P,2.40,5.1,2018-01-12,0.0131,6750.212857,950.212857,6751.233067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10893,2022-11-11,2022-12-09,14600.0,C,0.20,2.60,2022-12-09,P,2740.00,2764.7,2022-11-11,0.0371,11804.796038,-2795.203962,11805.333099
10894,2022-11-11,2022-12-09,14700.0,C,0.10,2.40,2022-12-09,P,2839.40,2864.2,2022-11-11,0.0371,11804.892928,-2895.107072,11805.333099
10895,2022-11-11,2022-12-09,14800.0,C,0.05,2.25,2022-12-09,P,2939.00,2963.8,2022-11-11,0.0371,11804.889819,-2995.110181,11805.333099
10896,2022-11-11,2022-12-09,14900.0,C,0.00,2.10,2022-12-09,P,3038.60,3063.3,2022-11-11,0.0371,11804.936710,-3095.063290,11805.333099


In [27]:
merged_data["adjusted_strike"] = np.round(merged_data["strike"]/merged_data["implied_S_y"],2)
merged_data

,quote_date,expiration_C,strike,option_type_C,bid_1545_C,ask_1545_C,expiration_P,option_type_P,bid_1545_P,ask_1545_P,date,rate,implied_S_x,moneyness,implied_S_y,adjusted_strike
0,2018-01-12,2018-02-09,5400.0,C,1347.00,1368.30,2018-02-09,P,0.05,3.1,2018-01-12,0.0131,6750.221453,1350.221453,6751.233067,0.80
1,2018-01-12,2018-02-09,5500.0,C,1248.00,1269.00,2018-02-09,P,0.25,3.7,2018-01-12,0.0131,6750.563054,1250.563054,6751.233067,0.81
2,2018-01-12,2018-02-09,5600.0,C,1151.60,1165.90,2018-02-09,P,0.65,4.2,2018-01-12,0.0131,6750.254655,1150.254655,6751.233067,0.83
3,2018-01-12,2018-02-09,5700.0,C,1052.40,1066.70,2018-02-09,P,1.70,4.3,2018-01-12,0.0131,6750.371256,1050.371256,6751.233067,0.84
4,2018-01-12,2018-02-09,5800.0,C,953.10,967.40,2018-02-09,P,2.40,5.1,2018-01-12,0.0131,6750.212857,950.212857,6751.233067,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10893,2022-11-11,2022-12-09,14600.0,C,0.20,2.60,2022-12-09,P,2740.00,2764.7,2022-11-11,0.0371,11804.796038,-2795.203962,11805.333099,1.24
10894,2022-11-11,2022-12-09,14700.0,C,0.10,2.40,2022-12-09,P,2839.40,2864.2,2022-11-11,0.0371,11804.892928,-2895.107072,11805.333099,1.25
10895,2022-11-11,2022-12-09,14800.0,C,0.05,2.25,2022-12-09,P,2939.00,2963.8,2022-11-11,0.0371,11804.889819,-2995.110181,11805.333099,1.25
10896,2022-11-11,2022-12-09,14900.0,C,0.00,2.10,2022-12-09,P,3038.60,3063.3,2022-11-11,0.0371,11804.936710,-3095.063290,11805.333099,1.26


In [28]:
merged_data['adjusted_strike'].value_counts()

0.99    302
1.00    295
1.02    289
1.01    287
0.98    282
       ... 
1.49      1
1.51      1
1.52      1
1.53      1
1.54      1
Name: adjusted_strike, Length: 105, dtype: int64

In [33]:
def adj_strike_min(group):
    return group['adjusted_strike'].min()

def adj_strike_max(group):
    return group['adjusted_strike'].max()

subset_min = merged_data.groupby("quote_date").apply(adj_strike_min).max()

subset_max = merged_data.groupby("quote_date").apply(adj_strike_max).min()

1.09

In [24]:
merged_data.to_csv("DATA.csv")